# week2 Assignment

## 邮件结构

- 将邮件的存储格式分为以下几部分
   - 'Metadata': 这部分包含从MIME头提取的信息, 如'From', 'Receive', 'Send_time', 'Subject'等等
   - 'Content': 这部分会包含所有的内容信息, 如'Body_text', 'Body_html', 'Recite', 'Attachment', 'Signature'等等
   - 'Entities': 包含邮件中提到的各种实体, 如'Name', 'Organization', 'Time', 'Position', 'Tel'
   - 'Relation': 包含邮件内的各种关系, 如邮件之间的关系, 邮件内容的语义关系.

## 思路

- 利用flanker提取MIME头, 将信息初步提取, 这是可以完成'Metadata'部分信息的提取
- 利用Regex将邮件内容分解, 段落, 引用, 附件, 签名档一次提取出来.
- 利用NLTK, jieba等分词工具, 进一步细化, 提取各个实体.
- 最后进行更深入的关系分析提取.
- 这样层层递进, 逐渐深入,


## 难点

- 分段有可能比较混乱, 这里可能会花一点时间
- 引用通过'>','>>'来判断
- 签名档因为比较复杂, 格式不一, 甚至有的没有, 有的特别简单, 信息不够全面
- 关系表示, 邮件内部, 邮件外部

## Tips

- 邮件的结尾都是--boundary--


In [5]:
import os
import re
import json
import flanker

from bs4 import BeautifulSoup
from flanker import mime

### 读取数据

In [117]:
def prepare_data(filename='2013-11.mbx'):
    with open(filename, 'r') as f:
        data = f.read()
    f.close()
    data_list = filter(None, re.split(r'From\s([\w+.?]+@(\w+\.)+(\w+))', data))  #  
    # Here I have to add twice for-loop, I haven't analyse the reason
    for data in data_list:
        if len(str(data)) < 500:
            data_list.remove(data)
    for data in data_list:
        if len(str(data)) < 500:
            data_list.remove(data)
    return data_list  

In [118]:
data_list = prepare_data('2013-11.mbx')

In [120]:
def extract_headers(msg_string):
    mime_dict = {}
    msg = mime.from_string(msg_string)
    msg_list = msg.headers.items()
    mime_keys = ['From', 'Date', 'Cc', 'To', 'Subject', ]
    for item in msg_list:
        if item[0] in mime_keys:
            mime_dict[item[0]] = item[1]
    return mime_dict

In [121]:
extract_headers(data_list[3])

{'Cc': u'\u4e2d\u6587HTML5\u540c\u6a02\u6703ML <public-html-ig-zh@w3.org>',
 'Date': u'Wed, 6 Nov 2013 20:09:11 +0800',
 'From': u'\u8463\u798f\u8208 Bobby Tung <bobbytung@wanderer.tw>',
 'Subject': u'Re: \u95dc\u65bc<cite>\u5143\u7d20\u6700\u8fd1\u7684\u5b9a\u7fa9\u4fee\u6539',
 'To': u'Yijun Chen <ethantw@me.com>'}

In [33]:
msg = mime.from_string(data_list[3])
for part in msg.parts:
    print 'Content-Type: {}, Body: {}'.format(part, part.body)

Content-Type: (text/plain), Body: Hi friends,

In light of the upcoming TPAC, I'd like to suggest a joint meeting between the two IGs. 

There have been some discussion in the Chinese IG on things related to publishing and I thought it will be nice for us to catch up with the Digital Publishing IG.

Agenda

0. Mutual introduction
1. CSS3 text (some discussion on our side)
2. digital publishing requirement for chinese language (Bobby has written a spec/requirement and it'd be nice to know how everyone thinks)
3. anything else?

This discussion won't be an exhaustive one, rather it is to put names to faces, discuss the agendas, and hopefully drive future online discussions.

If everyone is cool, maybe we can do a 90 minute on Thursday during TPAC? 

---
Zi Bin Cheah
HTML5 Chinese IG chair



Content-Type: (text/html), Body: <html><head></head><body style="word-wrap: break-word; -webkit-nbsp-mode: space; -webkit-line-break: after-white-space; ">Hi friends,<br><br>In light of the upcoming 

In [138]:
def extract_content(message_string):
    content_dict = {}
    msg = mime.from_string(message_string)   
    for part in msg.parts:
        content_dict[str(part)] = part.body
    return content_dict

In [146]:
content_dict = extract_content(data_list[3])